<a href="https://colab.research.google.com/github/krzysztofnowakuz/AGH/blob/main/Decyclization_%26_Quick_heuristic_recovery_policy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
#Wczytywanie pliku mainForm.txt to plik, w którym zawarto macierz incydencji uzupełnionej o znakowanie początkowe
data=pd.read_csv("mainForm.txt", sep=" ", header=None)
#Dodanie znakowania do macierzy incydencji(zabieg typowo kosmetyczny, aby ułatwić czytelność)
tabCol=[]
tabRow=[]

for col in data.columns:
    tabCol.append( "P" + str(col+1))
data.columns = tabCol

for row in data.index-2:
    tabRow.append("T" + str(row+1))
data.index=tabRow

#Zamiana wartości "x" na -1.
incidenceMatrix = data.replace('x', -1)

#Wydobycie znakowania początkowego z danych.
initialMarking = incidenceMatrix.head(2)
initialMarking = initialMarking.to_numpy()

#Konwersja danych do int oraz usunięcie znakowania początkowego z danych, aby pozostawić czystą macierz incydencji.
incidenceMatrix = incidenceMatrix[2:]
incidenceMatrix = incidenceMatrix.astype(int)

numEdges, numVertices = incidenceMatrix.shape

#Incidence matrix

In [2]:
print(incidenceMatrix)

    P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11
T1  -1   1   0   0   0   0   0   0  -1    0    0
T2   0  -1   1   0   0   0   0   0   1   -1    0
T3   0   0  -1   1   0   0   0   0   0    1   -1
T4   1   0   0  -1   0   0   0   0   0    0    1
T5   0   0   0   0   1   0   0  -1   0    0   -1
T6   0   0   0   0  -1   1   0   0   0   -1    1
T7   0   0   0   0   0  -1   1   0  -1    1    0
T8   0   0   0   0   0   0  -1   1   1    0    0


#Initial marking

In [3]:
print(initialMarking)

[['i' 'a' 'a' 'a' 'a' 'a' 'a' 'i' 'r' 'r' 'r']
 ['3' '0' '0' '0' '0' '0' '0' '3' '1' '1' '1']]


In [4]:
#Obliczenie wszystkich wierzchołków oraz wydobycie z macierzy incydencji listy krawędzi grafu skierowanego.
#Wierzchołki = ilość wierzchołków wydobywana z ilości kolumn w macierzy incydencji.
vertices = numVertices

#Wygenerowanie listy wierzchołków od 1 do vertices.
vertices = list(range(1, vertices + 1))

#Pusta lista na krawędzie grafu.
edges = []

#Iterujemy po wszystkich wierszach, aby wyszukać wszystkie tails i heads, a następnie stworzyć z nich parę reprezentującą krawędź i dodajemy ją do listy edges.
for row in incidenceMatrix.values:
    tails = [i+1 for i, x in enumerate(row) if x == -1]
    heads = [i+1 for i, x in enumerate(row) if x == 1]
    edges.extend([(tail, head) for tail in tails for head in heads])

print("Vertices =", vertices)
print("Edges =", edges)

Vertices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Edges = [(1, 2), (9, 2), (2, 3), (2, 9), (10, 3), (10, 9), (3, 4), (3, 10), (11, 4), (11, 10), (4, 1), (4, 11), (8, 5), (11, 5), (5, 6), (5, 11), (10, 6), (10, 11), (6, 7), (6, 10), (9, 7), (9, 10), (7, 8), (7, 9)]


In [5]:
#Tworzymy dwie listy na head i tail, gdzie head = początek krawędzi, a tail koniec krawędzi.
head = [e[0] for e in edges]
tail = [e[1] for e in edges]

for e in edges:
    head[e[0]] += 1
    tail[e[1]] += 1

#Obliczanie wagi dla każdej krawędzi bazując na ilości wystąpień głów i ogonów.
#Wzór: liczba wystąpień ogona krawędzi e jako ogona - liczba wystąpień ogona e jako głowy + liczba wystąpień głowy krawędzi e jako głowy - liczba wystąpień głowy krawędzi e jako ogona.
w = {}
for e in edges:
    w[e] = tail[e[1]] - head[e[1]] + head[e[0]] - tail[e[0]]

In [6]:
#Sortowanie w sposób nie malejący wszystkich wierzchołków na podstawie wyliczonych wcześniej wag.
sorted_edges = sorted(w.items(), key=lambda x: x[1])
for edge, weight in sorted_edges:
    print(f'Edge: {edge}, Weight: {weight}')

Edge: (3, 4), Weight: -14
Edge: (11, 4), Weight: -14
Edge: (7, 8), Weight: -14
Edge: (3, 10), Weight: -10
Edge: (11, 10), Weight: -10
Edge: (11, 5), Weight: -8
Edge: (7, 9), Weight: -8
Edge: (6, 10), Weight: -4
Edge: (2, 9), Weight: -2
Edge: (9, 10), Weight: -2
Edge: (4, 1), Weight: 0
Edge: (9, 2), Weight: 2
Edge: (10, 9), Weight: 2
Edge: (5, 6), Weight: 2
Edge: (10, 6), Weight: 4
Edge: (2, 3), Weight: 6
Edge: (8, 5), Weight: 6
Edge: (6, 7), Weight: 6
Edge: (1, 2), Weight: 8
Edge: (5, 11), Weight: 8
Edge: (9, 7), Weight: 8
Edge: (10, 3), Weight: 10
Edge: (10, 11), Weight: 10
Edge: (4, 11), Weight: 14


In [7]:

#Poniższa funkcja wykorzystując BFS sprawdza czy istnieje ścieżka od wierzchołka start do wierzchołka end w danym grafie.
#visited - wierzchołki odwiedzone podczas przeszukiwania.
#queue - wierzchołki do odwiedzenia
def is_path_exists(graph, start, end):
    visited = set()
    queue = [start]

    while queue: #Pętla wykonuje się dopóki kolejka nie jest pusta.
        vertex = queue.pop(0) #Usunięcie i zwrócenie pierwszego elementu z kolejki
        if vertex == end:
            return True  #Jeśli obecny wierzchołek to wierzchołek końcowy, to zwracamy True
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph.get(vertex, []))  #Jeśli rozpatrywany wierzchołek nie został jeszcze odwiedzony dodajemy go do zbioru odwiedzonych i dodajemy wszystkich nieodwiedzonych sąsiadów do kolejki.
    return False
#E_prime to lista krawędzi, które są dodawane do grafu G_prime.
G_prime = {}
E_prime = []

for edge, weight in sorted_edges:
    head, tail = edge
    if not is_path_exists(G_prime, head, tail):
        E_prime.append(edge)
        if head in G_prime:
            G_prime[head].append(tail)
        else:
            G_prime[head] = [tail]
print("E'",E_prime)
print("G'",G_prime)

E' [(3, 4), (11, 4), (7, 8), (3, 10), (11, 10), (11, 5), (7, 9), (6, 10), (2, 9), (9, 10), (4, 1), (9, 2), (10, 9), (5, 6), (10, 6), (2, 3), (8, 5), (6, 7), (1, 2), (5, 11)]
G' {3: [4, 10], 11: [4, 10, 5], 7: [8, 9], 6: [10, 7], 2: [9, 3], 9: [10, 2], 4: [1], 10: [9, 6], 5: [6, 11], 8: [5], 1: [2]}
